In [1]:
#load libraries
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 10)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [41]:
#Add columns
unscaled_data <- read_csv("pulsar_data.csv", 
                        col_names = c("mean_integrated_profile", 
                                      "stand_dev_integrated_profile", 
                                      "exc_kurtosis_integrated_profile", 
                                      "skew_integrated_profile",
                                      "mean_dmsnr", 
                                      "stand_dev_dmsnr", 
                                      "exc_kurtosis_dmsnr", 
                                      "skew_dmsnr", "class")) 

unscaled_data <- unscaled_data |>
    mutate(class = as_factor(class))
#pulsar_data 

#Class proportions in pulsar dataset (Imbalanced)
nrow(unscaled_data)
pulsar_proportions <- unscaled_data |>
    group_by(class) |>
    summarize(n = n()) |>
    mutate(percent = 100*n/nrow(pulsar_data))
pulsar_proportions

pulsar_train <- unscaled_data |>
    select(class, skew_dmsnr, skew_integrated_profile) 
#pulsar_train

pulsar_split <- initial_split(unscaled_data, prop = 0.75, strata = class)
pulsar_train <- training(pulsar_split)
pulsar_test <- testing(pulsar_split) 

pulsar_recipe <- recipe(class ~ ., data = pulsar_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors()) |>
    prep()
#pulsar_recipe

pulsar_data <- bake(pulsar_recipe, pulsar_train)
#scaled_pulsar

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 5) |>
  set_engine("kknn") |>
  set_mode("classification")
#knn_spec

knn_fit <- workflow() |>
  add_recipe(pulsar_recipe) |>
  add_model(knn_spec) |>
  fit(data = pulsar_train)
#knn_fit

pulsar_test_predictions <- predict(knn_fit, pulsar_test) |>
    bind_cols(pulsar_test)
#pulsar_test_predictions

pulsar_test_predictions |>
  metrics(truth = class, estimate = .pred_class) |>
  filter(.metric == "accuracy")
pulsar_test_predictions

Rows: 17898 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): mean_integrated_profile, stand_dev_integrated_profile, exc_kurtosis...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 17898

class,n,percent
<fct>,<int>,<dbl>
0,16259,121.12791
1,1639,12.21039


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9765363


.pred_class,mean_integrated_profile,stand_dev_integrated_profile,exc_kurtosis_integrated_profile,skew_integrated_profile,mean_dmsnr,stand_dev_dmsnr,exc_kurtosis_dmsnr,skew_dmsnr,class
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
0,140.5625,55.68378,-0.23457141,-0.699648398,3.1998328,19.110426,7.975532,74.2422249,0
0,103.0156,39.34165,0.32332837,1.051164429,3.1212375,21.744669,7.735822,63.1719091,0
0,138.1797,51.52448,-0.03185233,0.046797173,6.3302676,31.576347,5.155940,26.1433102,0
0,109.6406,49.01765,0.13763583,-0.256699775,1.5083612,12.072901,13.367926,223.4384192,0
0,136.0938,51.69100,-0.04590893,-0.271816393,9.3428094,38.096400,4.345438,18.6736485,0
0,105.4453,41.13997,0.14265380,0.320419676,3.5518395,20.755017,7.739552,68.5197706,0
0,117.3672,53.90861,0.25795344,-0.405049077,6.0183946,24.766123,4.807783,25.5226156,0
0,106.6484,56.36718,0.37835507,-0.266371607,2.4364549,18.405371,9.378660,96.8602254,0
0,112.7188,50.30127,0.27939095,-0.129010712,8.2817726,37.810012,4.691827,21.2762098,0
